In [5]:
include("/home/abhijith/Dropbox/Code/GRISE_code/MCMC.jl")
include("/home/abhijith/Dropbox/Code/GRISE_code/Potts_learning.jl")
using IterTools:product

In [6]:

function pth_order_tensor(r)
    a =  Tuple([2 for i in 1:r])
    M =  zeros(a)
    for i in CartesianIndices(M)
        s = prod(2*[Tuple(i)...] .- 3)
        M[i] = s
    end
    return M
end

pth_order_tensor (generic function with 1 method)

In [7]:
n =15
q = 2
#p = 4
avg_d =  3
prob = avg_d/(n-1)

Terms = Dict{}()



M =  pth_order_tensor(2)

for i in 1:n-1
    Terms[(i,i+1)] = (0.3 + rand())*M
    
end

Terms[(1,n)] = (0.3 + rand())*M
 
Terms[(1,3,5,7,9)] = 0.5*pth_order_tensor(5)
 



F = FactorGraph([2,5],n,q,Terms);
F

edge_list =  collect(keys(Terms))

16-element Array{Any,1}:
 (14, 15)       
 (2, 3)         
 (13, 14)       
 (5, 6)         
 (10, 11)       
 (1, 15)        
 (8, 9)         
 (12, 13)       
 (4, 5)         
 (7, 8)         
 (9, 10)        
 (6, 7)         
 (1, 2)         
 (1, 3, 5, 7, 9)
 (11, 12)       
 (3, 4)         

In [8]:
#Sample from model and save at path
n_samples =10^6

s = raw_sampler_potts(F, n_samples, false)
samples=[2*a .- 3  for a in s]
sample_path ="saved_model/samples.csv"
writedlm( sample_path,hcat(samples...))
#println("exact", countmap(samples))
edge_path = "saved_model/edges.csv" 
writedlm(edge_path, edge_list)

In [35]:
path_list= []
H=[]
activation_fn_list = [ swish, identity, swish]
for i in 1:n
    push!(path_list, "saved_model/model$i.h5" )
    path =  "saved_model/model$i.h5"
    c =  h5open(path, "r")
        wts_biases = [ collect( values(read(a)) ) for a in c]
        layers = []
        for (k,l) in enumerate(wts_biases)
#            println(k)
            W =  l[1]["kernel:0"]
            b =  l[1]["bias:0"]

                push!(layers, Dense(W,b, activation_fn_list[k]))
        end
        push!(H, Chain(layers[3], layers[1], layers[2]))
    
end

H

10-element Array{Any,1}:
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))

In [64]:
path_list= []
H=[]
for i in 1:n
    push!(path_list, "saved_model/model$i.h5" )
    path =  "saved_model/model$i.h5"
    end    
H =  load_model(path_list, [swish, swish, identity])

10-element Array{Any,1}:
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))
 Chain(Dense(9, 10, swish), Dense(10, 10, swish), Dense(10, 1))

UndefVarError: UndefVarError: H not defined

In [56]:
a =  fill([1,-1], n-1)
S = [collect(product(a...))...]

poly_weights = []



for r in 1:n-1
 max_f_r = 0
for i in 1:n-1
    
    for K in permutations(1:n-1, r)
            f_K = 0
        for s in S
            f_K += prod(s[[K...]])*H[i]([s...])[1]
        end
            max_f_r = max(max_f_r, abs(f_K/2^(n-1)))
    end
    
end
    push!(poly_weights, max_f_r)
end
poly_weights

┌ Warning: `product(xss...)` is deprecated, use `Iterators.product(xss...)` instead.
│   caller = top-level scope at In[56]:2
└ @ Core In[56]:2


9-element Array{Any,1}:
 0.09603820914014215  
 0.42072929802023573  
 0.5707318269506407   
 0.052029860095674785 
 0.04138361785422131  
 0.04127160513071157  
 0.013415225145339757 
 0.007310459591334005 
 0.0037574052441541337

In [65]:
est_samples = Gibbs_sampler_Ising(n, n_samples, 100000, H)
TVD(countmap(samples), countmap(est_samples), n_samples)

0.0116125

In [55]:
#test conditional probabilities
a =  fill([1,2], n-1)
err = 0
count =  0
for u in 1:n
    indices =  deleteat!(collect(1:n),u)
    for sa in product(a...)
        sa = [sa...]
        Z1 = 0
        Z2 = 0
        for i in 1:2  
            insert!(sa, u , i)
            Z1 += exp(conditional_energy(u, [sa...], F, true ))
            sa1=[2*a .- 3  for a in sa]
            Z2 += exp(sa1[u]*H[u](sa1[indices])[1])
            deleteat!(sa, u)
            
        end
        
        for i in 1:2  
            insert!(sa, u , i)
        
            p1 = exp(conditional_energy(u, [sa...], F, true ))/Z1
            sa1=[2*a .- 3  for a in sa]
            p2 = exp(sa1[u]*H[u](sa1[indices])[1])/Z2
            #println(Z2," " ,H[u](sa[indices])[1]) 
            deleteat!(sa, u)
            err += abs(p1-p2)
            count +=1
        end

    end
end

println(err/count)

┌ Warning: `product(xss...)` is deprecated, use `Iterators.product(xss...)` instead.
│   caller = top-level scope at In[55]:7
└ @ Core In[55]:7


0.026846338653487612


In [26]:
#Grise with prior info about the graph
edge_list =  collect(keys(F.terms))
M = optimizer_graph(s,2, [2], edge_list)
H_GRISE =  extract_couplings_graph(M, n, 2, [2], edge_list)

Total number of variables............................:      714
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        8
                     variables with only upper bounds:        0
Total number of equality constraints.................:      706
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0


Number of Iterations....: 6

                                   (scaled)                 (unscaled)
Objective...............:   3.5529306837179792e-01    3.5529306837179792e-01
Dual infeasibility......:   1.1823296986539963e-17    1.1823296986539963e-17
Constraint violation....:   8.3266726846886741e-17    8.3266726846886741e-17
Complementarity.........:   2.5059035603030268e-09    2.5059035603030268e-09
Overall NLP er

FactorGraph([2], 10, 2, Dict{Tuple,Array}((2, 3) => [0.5846707313785431 -0.5846707313785648; -0.5846707313785655 0.584670731378579],(6, 9) => [0.7118128426751738 -0.7118128426751482; -0.7118128426751484 0.7118128426751198],(4, 6) => [0.4549498627718622 -0.45494986277189503; -0.4549498627718948 0.4549498627719274],(8, 9) => [1.0916658019072512 -1.0916658019072183; -1.091665801907217 1.091665801907224],(2, 10) => [0.2506513375475682 -0.2506513375475448; -0.2506513375475448 0.25065133754755614],(5, 9) => [1.114569413464492 -1.1145694134645368; -1.1145694134645368 1.114569413464479],(1, 4) => [0.9746160648506446 -0.9746160648507254; -0.9746160648507245 0.9746160648506983],(2, 8) => [1.0442145186356753 -1.0442145186356697; -1.0442145186356693 1.0442145186356981],(5, 8) => [0.7041893725107822 -0.7041893725107735; -0.7041893725107732 0.7041893725107837],(4, 5) => [0.38146173001640243 -0.38146173001638123; -0.3814617300163818 0.38146173001644557]…))

In [27]:
F.terms

Dict{Tuple,Array} with 15 entries:
  (2, 3)  => [0.579775 -0.579775; -0.579775 0.579775]
  (6, 9)  => [0.718345 -0.718345; -0.718345 0.718345]
  (4, 6)  => [0.44088 -0.44088; -0.44088 0.44088]
  (8, 9)  => [1.10703 -1.10703; -1.10703 1.10703]
  (2, 10) => [0.250922 -0.250922; -0.250922 0.250922]
  (5, 9)  => [1.11266 -1.11266; -1.11266 1.11266]
  (1, 4)  => [0.980418 -0.980418; -0.980418 0.980418]
  (2, 8)  => [1.03478 -1.03478; -1.03478 1.03478]
  (5, 8)  => [0.697815 -0.697815; -0.697815 0.697815]
  (4, 5)  => [0.384562 -0.384562; -0.384562 0.384562]
  (7, 8)  => [0.27656 -0.27656; -0.27656 0.27656]
  (1, 8)  => [0.778697 -0.778697; -0.778697 0.778697]
  (2, 4)  => [0.683596 -0.683596; -0.683596 0.683596]
  (6, 7)  => [0.983908 -0.983908; -0.983908 0.983908]
  (3, 8)  => [0.294155 -0.294155; -0.294155 0.294155]